In [1]:
import numpy as np
import soundfile as sf
import librosa
import IPython.display as ipd

import barmuscomp.scripts.overall_scripts as scr
import barmuscomp.model.pattern_study as ps
import barmuscomp.model.features as features
import as_seg.data_manipulation as dm
from barmuscomp.model.current_plot import *
import barmuscomp.scripts.default_path as paths
import as_seg.barwise_input as bi

In [2]:
# Song
song_path = "C:/Users/amarmore/this_folder/The Beatles - Come Together.wav"
the_signal, sasmpling_rate = sf.read(song_path)
song_name = "The Beatles - Come Together"

bars = np.load("C:/Users/amarmore/Desktop/data_persisted/bars/The Beatles - Come Together.npy") #dm.get_bars_from_audio(song_path)

# STFT
Let's compute the STFT of the song:

In [3]:
n_fft=2048
hop_length = 32

#stft_complex = librosa.core.stft(np.asfortranarray(the_signal[:,0]), n_fft=n_fft, hop_length = hop_length)
#if the_signal.shape[1] > 1:
#    for i in range(1,the_signal.shape[1]):
#        stft_complex += librosa.core.stft(np.asfortranarray(the_signal[:,i]), n_fft=n_fft, hop_length = hop_length)

and then form the Time-Frequency-Bar tensor $\mathscr{X}$ of this STFT:

In [4]:
hop_length_seconds = hop_length / 44100
subdivision = 96

#complex_tensor_stft = bi.tensorize_barwise_FTB(stft_complex, bars, hop_length_seconds, subdivision)
#tensor_mag, tensor_phase = librosa.magphase(complex_tensor_stft, power=1) 

tensor_mag = np.load("C:/Users/amarmore/Desktop/data_persisted/cometogether/tensor_mag_ntd.npy", allow_pickle = True)
tensor_phase = np.load("C:/Users/amarmore/Desktop/data_persisted/cometogether/tensor_phase_ntd.npy", allow_pickle = True)

nb_bars_song = tensor_mag.shape[2]

As a baseline, we reconstruct the song from the unfolded tensor spectrogram. Hence, the song will be reconstructed from the 96 chosen samples per bar (subdivision parameter which defines the number of samples per bar, subsampling the previous over-sampled STFT).

To reconstruct the audio signal, Inverse STFT needs the hop length of the STFT. As bars can be of different length, and we subsampled 96 samples per bar, we compute the median hop length from the different bars, and applies it to all bars in our song.

In [5]:
median_hop = ps.get_median_hop(bars, subdivision)

Now, let's recreate the signal from the barwise STFT, in order to study the reconstruction quality of the Inverse STFT algorithm, and have a listenable baseline. We limit the song to a certain number of bars (not to overload the final HTML file).

In [6]:
#nb_bars_to_plot = 16 # you can set it to 89 if you use the executable format, and listen to the whole song.
#signal_stft_istft = librosa.istft(np.reshape((tensor_mag*tensor_phase)[:,:,:nb_bars], (1025, nb_bars_to_plot * subdivision), order = 'F'), hop_length = median_hop)
#ipd.Audio(signal_stft_istft, rate=sampling_rate)

We already hear some small artifacts from the reconstruction. Hence, reconstructed signals won't be better than this one, which is already disturbed.

# NTD: Nonnegative Tucker Decomposition
Let's compute the NTD of this TFB tensor, and study the reconstructed signal and the barwise patterns obtained in the decomposition.

As a recall, NTD is a tensor decomposition method, which can be used to retrieve patterns from data.

<img src="imgs/NTD.png" width="500"/>

We refer to [1,2] for details.

First, we need to set the dimensions of the decomposition, corresponding to the core dimensions. They are set empirically here.

In [7]:
ntd_dimensions = [32,12,10] #Dimensions of the decomposition
init_ntd = "tucker"

listen_audio = False
plot_patterns = False

# BETA 2

In [8]:
persisted_arguments = f"_{song_name}_stft_{init_ntd}_{subdivision}"
core_beta2, factors_beta2 = scr.NTD_decomp_as_script(paths.path_data_persisted_come_together, persisted_arguments, tensor_mag, ntd_dimensions, init = init_ntd, update_rule = "hals", beta = 2)

In [9]:
# Patterns Griffin-Lim
song_sdr_gl, patterns_sdr_gl, audio_song_ntd_gl, audio_patterns_ntd_gl = ps.sdr_songscale_patternscale_encpasulation(core_beta2, factors_beta2, median_hop, 
                                         tensor_mag_original = tensor_mag, tensor_phase_original = tensor_phase,
                                         phase_retrieval_song = "griffin_lim", phase_retrieval_patterns = "griffin_lim")
print("Griffin-Lim")

print(f"SDR at the songscale: {song_sdr_gl}")
print(f"SDR at the patternscale: avg = {np.mean(patterns_sdr_gl)}, std = {np.std(patterns_sdr_gl)}")

if listen_audio:
    ipd.display(audio_song_ntd_gl)
    for i in range(nb_patterns_to_show):
        ipd.display(audio_patterns_ntd_gl[i])

if plot_patterns:
    spec_patterns_ntd = ps.compute_patterns(core, factors[0], factors[1])
    for i in range(nb_patterns_to_show):
        plot_me_this_spectrogram(spec_patterns_ntd[i], title = f"{i}-th pattern")
        
# Patterns softmasking
song_sdr_mask, patterns_sdr_mask, audio_song_ntd_mask, audio_patterns_ntd_mask = ps.sdr_songscale_patternscale_encpasulation(core_beta2, factors_beta2, median_hop, 
                                         tensor_mag_original = tensor_mag, tensor_phase_original = tensor_phase,
                                         phase_retrieval_song = "original_phase", phase_retrieval_patterns = "softmasking")
print("Softmasking")
print(f"SDR at the songscale: {song_sdr_mask}")
print(f"SDR at the patternscale: avg = {np.mean(patterns_sdr_mask)}, std = {np.std(patterns_sdr_mask)}")

if listen_audio:
    ipd.display(audio_song_ntd_mask)
    for i in range(nb_patterns_to_show):
        ipd.display(audio_patterns_ntd_mask[i])
        
if plot_patterns:
    for i in range(nb_patterns_to_show):
        plot_me_this_spectrogram(spec_patterns_ntd[i], title = f"{i}-th pattern")

Griffin-Lim
SDR at the songscale: -38.46780219111554
SDR at the patternscale: avg = -20.814493857782285, std = 2.437747072404799
The current masked spectrogram is too close to the ogirinal spectrogram, the SDR would be disinformative.
The current masked spectrogram is too close to the ogirinal spectrogram, the SDR would be disinformative.
Masking
SDR at the songscale: 4.350775455906698
SDR at the patternscale: avg = 16.70731643232073, std = 5.101187884719196


# Beta 1

In [10]:
persisted_arguments = f"mu_slow_{song_name}_beta1_stft_{init_ntd}_{subdivision}_n_iter_max1000"
core_beta1, factors_beta1 = scr.NTD_decomp_as_script(paths.path_data_persisted_come_together, persisted_arguments, tensor_mag, ntd_dimensions, init = init_ntd, update_rule = "mu", beta = 1)

In [11]:
# Patterns Griffin-Lim
song_sdr_gl, patterns_sdr_gl, audio_song_ntd_gl, audio_patterns_ntd_gl = ps.sdr_songscale_patternscale_encpasulation(core_beta1, factors_beta1, median_hop, 
                                         tensor_mag_original = tensor_mag, tensor_phase_original = tensor_phase,
                                         phase_retrieval_song = "griffin_lim", phase_retrieval_patterns = "griffin_lim")
print("Griffin-Lim")
print(f"SDR at the songscale: {song_sdr_gl}")
print(f"SDR at the patternscale: avg = {np.mean(patterns_sdr_gl)}, std = {np.std(patterns_sdr_gl)}")

if listen_audio:
    ipd.display(audio_song_ntd_gl)
    for i in range(nb_patterns_to_show):
        ipd.display(audio_patterns_ntd_gl[i])

if plot_patterns:
    spec_patterns_ntd = ps.compute_patterns(core, factors[0], factors[1])
    for i in range(nb_patterns_to_show):
        plot_me_this_spectrogram(spec_patterns_ntd[i], title = f"{i}-th pattern")

# Patterns softmasking
song_sdr_mask, patterns_sdr_mask, audio_song_ntd_mask, audio_patterns_ntd_mask = ps.sdr_songscale_patternscale_encpasulation(core_beta1, factors_beta1, median_hop, 
                                         tensor_mag_original = tensor_mag, tensor_phase_original = tensor_phase,
                                         phase_retrieval_song = "original_phase", phase_retrieval_patterns = "softmasking")
print("Softmasking")
print(f"SDR at the songscale: {song_sdr_mask}")
print(f"SDR at the patternscale: avg = {np.mean(patterns_sdr_mask)}, std = {np.std(patterns_sdr_mask)}")

if listen_audio:
    ipd.display(audio_song_ntd_mask)
    for i in range(nb_patterns_to_show):
        ipd.display(audio_patterns_ntd_mask[i])
        
if plot_patterns:
    for i in range(nb_patterns_to_show):
        plot_me_this_spectrogram(spec_patterns_ntd[i], title = f"{i}-th pattern")

Griffin-Lim
SDR at the songscale: -34.52657827635712
SDR at the patternscale: avg = -17.68561909189689, std = 3.0296033687002533
Masking
SDR at the songscale: 6.077240279730299
SDR at the patternscale: avg = 25.942390346762654, std = 5.481567223668133


# BETA 0

In [12]:
persisted_arguments = f"mu_slow_{song_name}_beta0_stft_{init_ntd}_{subdivision}_n_iter_max1000"
core_beta0, factors_beta0 = scr.NTD_decomp_as_script(paths.path_data_persisted_come_together, persisted_arguments, tensor_mag, ntd_dimensions, init = init_ntd, update_rule = "mu", beta = 0)

In [13]:
# Patterns Griffin-Lim
song_sdr_gl, patterns_sdr_gl, audio_song_ntd_gl, audio_patterns_ntd_gl = ps.sdr_songscale_patternscale_encpasulation(core_beta0, factors_beta0, median_hop, 
                                         tensor_mag_original = tensor_mag, tensor_phase_original = tensor_phase,
                                         phase_retrieval_song = "griffin_lim", phase_retrieval_patterns = "griffin_lim")
print("Griffin-Lim")
print(f"SDR at the songscale: {song_sdr_gl}")
print(f"SDR at the patternscale: avg = {np.mean(patterns_sdr_gl)}, std = {np.std(patterns_sdr_gl)}")

if listen_audio:
    ipd.display(audio_song_ntd_gl)
    for i in range(nb_patterns_to_show):
        ipd.display(audio_patterns_ntd_gl[i])

if plot_patterns:    
    spec_patterns_ntd = ps.compute_patterns(core, factors[0], factors[1])
    for i in range(nb_patterns_to_show):
        plot_me_this_spectrogram(spec_patterns_ntd[i], title = f"{i}-th pattern")

# Patterns softmasking
song_sdr_mask, patterns_sdr_mask, audio_song_ntd_mask, audio_patterns_ntd_mask = ps.sdr_songscale_patternscale_encpasulation(core_beta0, factors_beta0, median_hop, 
                                         tensor_mag_original = tensor_mag, tensor_phase_original = tensor_phase,
                                         phase_retrieval_song = "original_phase", phase_retrieval_patterns = "softmasking")
print("Softmasking")
print(f"SDR at the songscale: {song_sdr_mask}")
print(f"SDR at the patternscale: avg = {np.mean(patterns_sdr_mask)}, std = {np.std(patterns_sdr_mask)}")

if listen_audio:
    ipd.display(audio_song_ntd_mask)
    for i in range(nb_patterns_to_show):
        ipd.display(audio_patterns_ntd_mask[i])
        
if plot_patterns:
    for i in range(nb_patterns_to_show):
        plot_me_this_spectrogram(spec_patterns_ntd[i], title = f"{i}-th pattern")

Griffin-Lim
SDR at the songscale: -36.990796615261424
SDR at the patternscale: avg = -20.47174234998537, std = 3.478175207131415
The current masked spectrogram is too close to the ogirinal spectrogram, the SDR would be disinformative.
Masking
SDR at the songscale: 2.511286202438921
SDR at the patternscale: avg = 19.01875056777578, std = 8.705962166649469
